In [35]:
import scqubits as scq
import numpy as np
import qutip as qt

In [127]:
fluxonium1 = scq.Fluxonium(
    EJ=4.6,
    EC=0.9,
    EL=0.21,
    cutoff = 20,
    flux = 0.5,
    truncated_dim=5
)
fluxonium2 = scq.Fluxonium(
    EJ=4.6,
    EC=0.9,
    EL=0.21,
    cutoff = 20,
    flux = 0.51,
    truncated_dim=5
)

In [128]:
evals = fluxonium2.eigenvals()
evals - evals[0]

array([0.        , 0.10062238, 4.44098152, 5.24896294, 7.57476678,
       7.78706112])

In [134]:
hamiltonian = fluxonium1.hamiltonian(energy_esys=True) - 0.21*fluxonium1.phi_operator(energy_esys=True)*0.01*2*np.pi
hamiltonian = qt.Qobj(hamiltonian)

In [135]:
evals, evecs = hamiltonian.eigenstates(tol=1e-10)
evals - evals[0]

array([0.        , 0.09913075, 4.44219508, 5.25154041, 7.61001674])

In [131]:
evals = fluxonium1.eigenvals()
evals - evals[0]

array([0.        , 0.06195051, 4.42502847, 5.23162226, 7.59119335,
       7.74587769])